# Plantilla ML

In [2]:
#encoding: utf-8
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from heapq import merge

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.preprocessing import Imputer,StandardScaler,MinMaxScaler
from sklearn.pipeline import make_pipeline

import pickle

from Tools import woe, woe_c, sus_woe, woe_l, metricas, iv,frecuencia, sust_woe_2




%matplotlib inline 

## Lectura de Datos

In [3]:
df=pd.read_csv("./Examen/train_examen.csv")

In [4]:
df.shape

(700, 21)

In [5]:
frecuencia(df, every=True)

  checking_status  Conteo  Frecuencia
0     no checking     282    0.402857
1              <0     192    0.274286
2        0<=X<200     181    0.258571
3           >=200      45    0.064286 


  duration  Conteo  Frecuencia
0     12.0     128    0.182857
1     24.0     125    0.178571
2     18.0      79    0.112857
3     36.0      61    0.087143
4      6.0      51    0.072857 


                   credit_history  Conteo  Frecuencia
0                   existing paid     369    0.527143
1  critical/other existing credit     197    0.281429
2              delayed previously      65    0.092857
3                        all paid      40    0.057143
4             no credits/all paid      29    0.041429 


               purpose  Conteo  Frecuencia
0             radio/tv     208    0.297143
1              new car     145    0.207143
2  furniture/equipment     127    0.181429
3             business      69    0.098571
4             used car      69    0.098571 


  credit_amount  Conteo  Frecu

In [6]:
with pd.option_context("display.max_rows",5,"display.max_columns", None ):
    display(df.head())

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,no checking,6.0,critical/other existing credit,new car,250.0,>=1000,1<=X<4,2.0,female div/dep/mar,none,2.0,real estate,41.0,bank,own,2.0,unskilled resident,1.0,none,yes,good
1,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,4.0,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes,good
2,0<=X<200,30.0,existing paid,furniture/equipment,3832.0,<100,<1,2.0,male mar/wid,none,1.0,life insurance,22.0,none,own,1.0,skilled,1.0,none,yes,good
3,0<=X<200,9.0,existing paid,new car,3195.0,no known savings,1<=X<4,1.0,female div/dep/mar,none,2.0,real estate,33.0,none,own,1.0,unskilled resident,1.0,none,yes,good
4,no checking,9.0,critical/other existing credit,new car,2134.0,<100,1<=X<4,4.0,male single,none,4.0,car,48.0,none,own,3.0,skilled,1.0,yes,yes,good


## Decisión de Variables

In [7]:
var_dist=["checking_status","credit_history","purpose", "savings_status", 
          "employment", "personal_status","other_parties",
          "property_magnitude", "other_payment_plans",
          "housing","job","own_telephone","foreign_worker"]

obj="class"
var_cont=list(set(df.columns)-(set(var_dist)|{obj}))

In [8]:
df[var_dist]=df[var_dist].astype(str)

## Binarización de Obj

In [9]:
df[obj]=df[obj].map(lambda x: 1 if x=="bad" else 0)

## IV

In [12]:
info_value=iv(df, obj, every=True, sm=True,bins=4)
info_value

[('existing_credits', 0.018261874158941743),
 ('foreign_worker', 0.03804337316498537),
 ('other_payment_plans', 0.06275339170357022),
 ('savings_status', 0.17640065373237157),
 ('checking_status', 0.7141917152784789),
 ('other_parties', 0.03526793434964368),
 ('credit_amount', inf),
 ('personal_status', 0.0525986701134156),
 ('residence_since', 0.024243316491213387),
 ('job', 0.002002876077607823),
 ('num_dependents', 0.00016241211436713614),
 ('property_magnitude', 0.11652563295537172),
 ('installment_commitment', 0.05362670153662853),
 ('duration', 0.19564049209687506),
 ('own_telephone', 0.012573440603243538),
 ('housing', 0.08706620104890492),
 ('age', 0.0647757531216292),
 ('employment', 0.10337026402445772),
 ('credit_history', 0.2957952600043244),
 ('purpose', 0.1785161304851879)]

## Eliminación de Variables no Predictivas o Sobre Predictivas

In [13]:
aux1=df.copy()


def del_iv(info_value):
    for i in range(len(info_value)):
        if info_value[i][1]<.02 or info_value[i][1]>.5:
            del aux1[info_value[i][0]]
            if info_value[i][0] in var_dist: var_dist.remove(info_value[i][0])
            elif info_value[i][0] in var_cont:var_cont.remove(info_value[i][0])


del_iv(info_value)

## Woe

In [14]:
woe_l(aux1, obj, var_dist)

=================checking_status==================
                    Woe
0<=X<200      39.882910
<0            89.689247
>=200        -71.796551
no checking -114.052941 


==================credit_history==================
                                       Woe
all paid                        111.579173
critical/other existing credit  -68.437456
delayed previously               21.133546
existing paid                     0.844577
no credits/all paid             145.536475 


=====================purpose======================
                            Woe
business              12.036368
domestic appliance    12.036368
education             44.578608
furniture/equipment   10.852922
new car               46.520417
other                 12.036368
radio/tv             -44.544408
repairs               81.351086
retraining          -113.239929
used car             -64.689147 


==================savings_status==================
                         Woe
100<=X<500         16.481544

In [15]:

for i in var_dist:
    aux1=sust_woe_2(df,aux1, obj, i)


In [16]:
aux1=aux1[df.columns]

In [17]:
aux1

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,-114.052941,6.0,-68.437456,46.520417,250.0,-109.603164,-4.777324,2.0,24.466140,1.727848,...,-51.634219,41.0,35.925559,-19.877334,2.0,0.788570,1.0,9.419396,3.237008,0
1,89.689247,12.0,111.579173,-44.544408,626.0,25.113391,-4.777324,4.0,24.466140,1.727848,...,-51.634219,24.0,35.925559,-19.877334,1.0,0.788570,1.0,9.419396,3.237008,1
2,-114.052941,21.0,0.844577,12.036368,1572.0,-109.603164,-26.262857,4.0,24.466140,1.727848,...,-51.634219,36.0,35.925559,-19.877334,1.0,0.788570,1.0,9.419396,3.237008,0
3,39.882910,42.0,-68.437456,12.036368,5954.0,25.113391,-30.637882,2.0,24.466140,1.727848,...,-51.634219,41.0,35.925559,-19.877334,2.0,0.788570,1.0,9.419396,3.237008,0
4,-71.796551,24.0,-68.437456,46.520417,1344.0,-58.235295,-30.637882,4.0,-16.495711,1.727848,...,-51.634219,37.0,35.925559,-19.877334,2.0,0.788570,2.0,9.419396,3.237008,1
5,39.882910,12.0,0.844577,-44.544408,1155.0,25.113391,-26.262857,3.0,-18.793768,-79.592705,...,-51.634219,40.0,35.925559,-19.877334,2.0,0.788570,1.0,9.419396,3.237008,0
6,39.882910,6.0,0.844577,-44.544408,484.0,25.113391,-30.637882,3.0,-18.793768,-79.592705,...,-51.634219,28.0,35.925559,-19.877334,1.0,0.788570,1.0,9.419396,3.237008,0
7,39.882910,12.0,0.844577,-44.544408,625.0,25.113391,56.006196,4.0,-18.793768,-79.592705,...,-51.634219,26.0,35.925559,-19.877334,1.0,0.788570,1.0,9.419396,3.237008,0
8,39.882910,24.0,-68.437456,10.852922,4736.0,25.113391,56.006196,2.0,24.466140,1.727848,...,8.799840,25.0,35.925559,-19.877334,1.0,0.788570,1.0,9.419396,3.237008,1
9,39.882910,12.0,0.844577,46.520417,888.0,25.113391,-26.262857,4.0,-16.495711,1.727848,...,8.799840,41.0,35.925559,-19.877334,1.0,0.788570,2.0,9.419396,3.237008,1


## Designación de Variables

In [18]:
var_all=list(set(df.columns)-{obj})
X=aux1[var_all]
y=aux1[obj]

## Partición de Entrenamiento

In [19]:
Xt,Xv,yt,yv =train_test_split(X,y,train_size=0.7)
for d in [Xt,Xv,yt,yv]:
    print d.shape

(489, 20)
(211, 20)
(489,)
(211,)


## Árbol de Decisión 

In [20]:
model_tree= DecisionTreeClassifier()
model_tree.fit(Xt,yt)
metricas(model_tree,Xt,Xv,yt,yv)

1.0
0.586029411765
1.0
0.63981042654
[[349   0]
 [  0 140]]
[[105  31]
 [ 45  30]]


In [23]:
param_grid_tree= dict(criterion=['gini','entropy'],
                 max_depth=range(1,21),
                 min_samples_split = range(100,210,10)
                 )

In [24]:
model_tree = DecisionTreeClassifier()
grid = GridSearchCV(cv=10,n_jobs=-1,scoring='roc_auc',
                    estimator=model_tree,
                    param_grid=param_grid_tree)
grid.fit(X,y)
print grid.best_estimator_
print grid.best_params_
print grid.best_score_

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=160, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
{'min_samples_split': 160, 'criterion': 'entropy', 'max_depth': 4}
0.72864696528


In [38]:
model_tree= grid.best_estimator_
model_tree.fit(Xt,yt)
metricas(model_tree,Xt,Xv,yt,yv)

0.780034793287
0.744460784314
0.756646216769
0.668246445498
[[347   2]
 [117  23]]
[[134   2]
 [ 68   7]]


## Regresión Logística 

In [53]:
model_log =LogisticRegression()
model_log.fit(Xt,yt)
metricas(model_log,Xt,Xv,yt,yv)

0.856528857962
0.769411764706
0.81799591002
0.706161137441
[[319  30]
 [ 59  81]]
[[117  19]
 [ 43  32]]


In [40]:
param_grid_log= dict(dual=[False, True],
                max_iter=range(100,210,10) )

In [54]:
model_log = LogisticRegression()
grid = GridSearchCV(cv=10,n_jobs=-1,scoring='roc_auc',
                    estimator=model_log,
                    param_grid=param_grid_log)
grid.fit(X,y)
print grid.best_estimator_
print grid.best_params_
print grid.best_score_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
{'max_iter': 100, 'dual': False}
0.767457814295


In [55]:
model_log= grid.best_estimator_
model_log.fit(Xt,yt)
metricas(model_log,Xt,Xv,yt,yv)

0.856528857962
0.769411764706
0.81799591002
0.706161137441
[[319  30]
 [ 59  81]]
[[117  19]
 [ 43  32]]


## Red Neuronal

In [76]:
model_red  = MLPClassifier()
model_red.fit(Xt,yt)
metricas(model_red,Xt,Xv,yt,yv)

0.72169463774
0.712352941176
0.736196319018
0.748815165877
[[296  53]
 [ 76  64]]
[[118  18]
 [ 35  40]]


In [77]:
imputer = Imputer(strategy='mean')
scaler = MinMaxScaler()
pca = PCA()
pipe = make_pipeline(imputer,pca,scaler)

In [78]:
pipe.fit(X)

Pipeline(steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1)))])

In [79]:
Xp=pd.DataFrame(pipe.transform(X),columns=['p%d'%x for x in range(1,len(var_all)+1)])

In [82]:
param_grid_red= dict(hidden_layer_sizes=[(i,) for i in range(5,21)],
activation = ['identity', 'logistic', 'tanh', 'relu'],
solver = ['lbfgs', 'sgd', 'adam'],
learning_rate = ['constant', 'invscaling', 'adaptive'],
max_iter = [1000])

In [83]:
%%time 
model_red = MLPClassifier()
grid = GridSearchCV(cv=4,n_jobs=-1,scoring='roc_auc',
                    estimator=model_red,
                    param_grid=param_grid_red)

grid.fit(Xp,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_

/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(7,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
{'activation': 'tanh', 'max_iter': 1000, 'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (7,)}
0.795906702002
CPU times: user 8.42 s, sys: 148 ms, total: 8.57 s
Wall time: 6min 22s


In [146]:
model_red= grid.best_estimator_
model_red.fit(Xt,yt)
metricas(model_red,Xt,Xv,yt,yv)

0.723444535407
0.720196078431
0.713701431493
0.644549763033
[[349   0]
 [140   0]]
[[136   0]
 [ 75   0]]


# Ensambles

In [147]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

### Bosque Aleatorio

In [148]:
param_grid_Ftree= dict(criterion=['gini','entropy'],
                 max_depth=range(1,11),
                 min_samples_split = range(100,210,10)
                 )

In [149]:
%%time 
model_Ftree= RandomForestClassifier()
grid = GridSearchCV(cv=4,n_jobs=-1,scoring='roc_auc',
                    estimator=model_Ftree,
                    param_grid=param_grid_Ftree)


grid.fit(X,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_
rf = grid.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=140, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
{'min_samples_split': 140, 'criterion': 'entropy', 'max_depth': 10}
0.771667966144
CPU times: user 1.96 s, sys: 68 ms, total: 2.03 s
Wall time: 11.9 s


### AdaBoost

In [150]:
param_grid_AB= dict(n_estimators = range(5,30),
learning_rate = np.arange(0.1,1.1,0.1))

In [151]:
%%time 
model_AB= AdaBoostClassifier()
grid = GridSearchCV(cv=4,n_jobs=-1,scoring='roc_auc',
                    estimator=model_AB,
                    param_grid=param_grid_AB)


grid.fit(X,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_
adab = grid.best_estimator_

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.70000000000000007, n_estimators=25,
          random_state=None)
{'n_estimators': 25, 'learning_rate': 0.70000000000000007}
0.770442651537
CPU times: user 1.14 s, sys: 52 ms, total: 1.19 s
Wall time: 13.9 s


### Voting Classifier

In [194]:
vc = VotingClassifier(estimators=[('adab', adab), ('rf', rf), ('log', model_log)],
                 voting='soft')

vc.fit(Xt,yt)

metricas(vc,Xt,Xv,yt,yv)

0.861277118297
0.779509803922
0.795501022495
0.715639810427
[[328  21]
 [ 79  61]]
[[125  11]
 [ 49  26]]


## Prueba consigo misma

In [189]:
s=vc
f=pd.DataFrame(s.predict(aux1[aux1[obj]==0][var_all]))
f.rename(columns={0:"0"}, inplace=True)

frecuencia(f, every=True)

   0  Conteo  Frecuencia
0  0     455    0.938144
1  1      30    0.061856 




## Prueba con Otros Datos

In [195]:
df2=pd.read_csv("./Examen/test_examen.csv")
aux2=df2.copy()

In [196]:
for i in var_dist:
    aux2=sust_woe_2(df, aux2, obj, i)
aux2=aux2[var_all]

In [197]:
aux2.shape

(300, 20)

In [203]:
model=vc
f=pd.DataFrame(model.predict(aux2))
f.rename(columns={0:"0"}, inplace=True)

frecuencia(f, every=True)

   0  Conteo  Frecuencia
0  0     246        0.82
1  1      54        0.18 




In [208]:
model=vc
d=pd.DataFrame(model.predict_proba(aux2))[1]
d.to_csv("./Examen/Probabilidad bad.csv", header=False, index=False)

0      0.258544
1      0.361285
2      0.643086
3      0.530326
4      0.251190
5      0.335397
6      0.334669
7      0.349046
8      0.415771
9      0.227242
10     0.315460
11     0.229365
12     0.254773
13     0.338417
14     0.219569
15     0.402784
16     0.206694
17     0.177478
18     0.189788
19     0.193127
20     0.230569
21     0.230427
22     0.545302
23     0.456209
24     0.212596
25     0.281998
26     0.344410
27     0.363233
28     0.394041
29     0.512549
         ...   
270    0.269437
271    0.479771
272    0.537902
273    0.382758
274    0.656273
275    0.396212
276    0.265226
277    0.461360
278    0.526013
279    0.343562
280    0.325386
281    0.643512
282    0.242661
283    0.545294
284    0.282710
285    0.209014
286    0.413084
287    0.331049
288    0.208297
289    0.433282
290    0.193317
291    0.205422
292    0.300014
293    0.205444
294    0.304260
295    0.225278
296    0.459069
297    0.313789
298    0.192003
299    0.497173
Name: 1, Length: 300, dt